In [1]:
import requests
import pandas as pd
import folium

def get_earthquakes():
    print("Fetching global seismic data from USGS...")
    # Fetch all earthquakes > M2.5 from the past day
    url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_day.geojson"
    data = requests.get(url).json()

    quakes = []
    for feature in data['features']:
        coords = feature['geometry']['coordinates']
        quakes.append({
            "Place": feature['properties']['place'],
            "Magnitude": feature['properties']['mag'],
            "Longitude": coords[0],
            "Latitude": coords[1],
            "Depth (km)": coords[2]
        })

    return pd.DataFrame(quakes)

def plot_quakes(df):
    m = folium.Map(location=[0, 0], zoom_start=2, tiles="CartoDB dark_matter")

    for _, row in df.iterrows():
        # Scale marker by earthquake magnitude
        mag = row['Magnitude']
        color = 'yellow' if mag < 4.5 else 'orange' if mag < 6.0 else 'red'

        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=mag * 1.5,
            popup=f"<b>{row['Place']}</b><br>Mag: {mag}",
            color=color, fill=True, fill_color=color, fill_opacity=0.7
        ).add_to(m)
    return m

# Run and display
quake_df = get_earthquakes()
display(quake_df.head())
display(plot_quakes(quake_df))

Fetching global seismic data from USGS...


,Place,Magnitude,Longitude,Latitude,Depth (km)
0,"92 km SW of Nikolski, Alaska",2.50,-169.8340,52.3610,7.300
1,"83 km SW of Nikolski, Alaska",2.60,-169.8060,52.4560,11.400
2,Kermadec Islands region,5.00,-179.7053,-31.2259,313.507
3,"9 km ESE of Ocotillo Wells, CA",2.64,-116.0505,33.0980,3.020
4,"2 km SE of Cathcart, Washington",3.02,-122.0700,47.8300,23.880
